In [1]:
from rainymotion import models, metrics, utils
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime,timedelta
import calendar
from netCDF4 import Dataset
import joblib
import os
print(os.getcwd())

/Users/jiang/github_all/rainymotion/docs/notebooks


In [2]:
kakuho_folder = "/Users/jiang/data/rain_kakuho"
radar_folder = "/Users/jiang/data/jma_radar"

In [9]:
time_step = 5 * 60 # seconds
threshold = 0.1  # mm/h
lead_time = list(range(0,65,5))

for j in range(0,24):
    dt_now = datetime(2020,1,28,j,0)
    dt_pre = dt_now - timedelta(seconds = time_step)
    dt_12  = dt_now + timedelta(seconds = time_step * 12)
    yday_pre = dt_pre.strftime('%Y_%m_%d')
    yday_now = dt_now.strftime('%Y_%m_%d')
    yday_12  = dt_12.strftime('%Y_%m_%d')

    # load ground truth data
    daily_database = {}
    inputs = np.zeros(shape = (2,1000,1000), dtype = np.float16)
    if yday_pre not in daily_database:
        daily_database[yday_pre] = joblib.load(os.path.join(radar_folder, 
                                                            f"jma_radar_uint8_{yday_pre}.joblib")) 
    if yday_12 not in daily_database:
        daily_database[yday_12]  = joblib.load(os.path.join(radar_folder,
                                                            f"jma_radar_uint8_{yday_12}.joblib")) 
    # load data into optical flow model
    sequence_in_day = int(dt_pre.hour*12 + (dt_pre.minute)/5)    
    inputs[0,:,:] = daily_database[yday_pre][sequence_in_day]/10.0
    inputs[1,:,:] = daily_database[yday_now][(sequence_in_day + 1)%288]/10.0  
    model = models.Dense()    
    model.input_data = inputs
    model.lead_steps = 13
    nowcast = model.run()  # shape (13, 1000, 1000)

    # load kakuho data from wgrib2 file
    grib_file = dt_now.strftime('%Y%m%d_%H%M00.000')
    source_folder = os.path.join(kakuho_folder, dt_now.strftime("%Y/%m/%d"))
    source_path = os.path.join(source_folder, grib_file)
    nc_file = source_path + ".nc"
    varlist = [":{} min".format(i) for i in range(5,65,5)]
    var = '|'.join(varlist)
    cmd = "wgrib2 {0} -s | egrep '({1})'|wgrib2 -i {0} -netcdf {2}".format(source_path, var, nc_file)
    fail = os.system(cmd)  # 0 indicate success, others indicate fail
    if fail:
        print("wgrib2 wrong at ", grib_file)
        continue

    root = Dataset(nc_file, "r")        
    # delete nc                     
    os.system(f"rm -r {nc_file}") 

    of_13_th = []  # threat score
    ka_13_th = [1]
    per_13_th = [1]
    coverage_13 = []
    for i in range (13):
        dt_fcst = dt_now + timedelta(seconds = time_step * i)
        yday_fcst = dt_fcst.strftime('%Y_%m_%d')
        sequence = int(dt_fcst.hour*12 + (dt_fcst.minute)/5)   
        ground_truth =  daily_database[yday_fcst][sequence]/10.0
        of_13_th.append(metrics.CSI(ground_truth, nowcast[i], threshold = threshold))
        coverage = np.sum(ground_truth >= threshold)/(1e6 - 132396)*100
        coverage_13.append(coverage)
        if i ==0:
            ground_truth0 = ground_truth
        if i >=1:
            rain_reduced = root['APCP_surface'][i-1,1500:2500,1000:2000]
            rain_reduced.fill_value = 0.0
            rain_filled = rain_reduced.filled().astype('float16') * 6 # mm/10 min to mm/h
            ka_13_th.append(metrics.CSI(ground_truth, rain_filled, threshold = threshold))
            per_13_th.append(metrics.CSI(ground_truth, ground_truth0, threshold = threshold))
    plt.figure(dpi=75)
    plt.plot(lead_time, of_13_th,'o-',label = "Optical Flow (rainymotion-dense)")
    plt.plot(lead_time, ka_13_th,'s-',label = "Kakuho")
    plt.plot(lead_time, per_13_th,'*-',label = "Persistence")
    plt.legend()
    plt.ylim([0,1])
    plt.grid()
    plt.ylabel("threat score")
    plt.xlabel("minutes after now")
    plt.title(f"now = {dt_now.strftime('%Y-%m-%d %H:%M')}, rain coverage = {coverage:.1f} %")
    plt.savefig(f"OFvsKakuho_{dt_now.strftime('%Y%m%d_%H%M')}.png",format = "png",bbox_inches='tight') 
    plt.close()

In [10]:
import imageio
png_files = glob.glob("./OFvsKakuho*.png")
png_files.sort()
print(len(png_files)) # 288

images = []
for filename in png_files:
    images.append(imageio.imread(filename))
    os.system(f"rm -r {filename}")
output_file = f'OFvsKakuho-{yday_now}.gif'
imageio.mimsave(output_file, images,duration = 1)  # unit is in seconds

24


In [1]:
import numpy as np

In [2]:
X = np.random.normal(size=(10, 3))

In [3]:
(X[i,:] for i in range(10))

<generator object <genexpr> at 0x10ac4bb30>